# Dataset Preparation

This notebook is used as a reference for the preparation of a dataset. From the given *CSV* files, we perform some processing operation to create the final dataset.

In [1]:
%reload_ext autoreload
%autoreload 2

import threadpoolctl
import numpy as np
import pandas as pd
from intellect.io import recursive_find_file, dump, load
from intellect.inspect import set_seed
from intellect.dataset import balance_classes, load_dataframes, remove_constant_columns, format_dataframe_columns_names, format_dataframe_columns_values

threadpoolctl.threadpool_limits(limits=2);

Define the parameters. Note that these define the outcome of this notebook. Each parameter is explained in the comments.

In [2]:
# Parameters

# path to the CICIDS2017 and CICIDS2019 datasets
DATASET_2017 = "../../datasets/CICIDS2017/"
DATASET_2019 = "../../datasets/CICIDS2019/"

# column marked as label in these datasets
LABEL = 'Label'

# labels to be considered as benign (class=0 in binary classification)
BENIGN_LABELS = ['BENIGN']

# columns to remove from datasets (session identifiers and non-commond features)
EXCLUDED_COLUMNS = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Protocol', 'Timestamp', 'SimillarHTTP', 'Inbound']

OUTPUT_DIR = "./"

At first, load only the label column for both the two datasets

In [3]:
set_seed()
files2019 = recursive_find_file(DATASET_2019, endswith_condition=".csv")
frames_only_labels2019 = load_dataframes(files2019, only_labels_str=LABEL)

In [4]:
files2017 = recursive_find_file(DATASET_2017, endswith_condition=".csv")
frames_only_labels2017 = load_dataframes(files2017, only_labels_str=LABEL)

Keep track of the original file and index (row in the file) to which each label comes from.
Insert to each label the string "2017" or "2019" depending on their source dataset.

In [5]:
df = []
for name, (dictionary) in zip(["2017", "2019"], [frames_only_labels2017, frames_only_labels2019]):
    for k, v in dictionary.items():
        v["File"] = k
        v["Indexes"] = v.index.values
        v[LABEL] += f"-{name}"
        df.append(v)
df = pd.concat(df)

Print distribution of labels.

In [6]:
df[LABEL].value_counts()

Label
TFTP-2019                          20082580
Syn-2019                            6473789
MSSQL-2019                          5787453
DrDoS_SNMP-2019                     5159870
DrDoS_DNS-2019                      5071011
DrDoS_MSSQL-2019                    4522492
DrDoS_NetBIOS-2019                  4093279
UDP-2019                            3867155
NetBIOS-2019                        3657497
DrDoS_UDP-2019                      3134645
DrDoS_SSDP-2019                     2610611
BENIGN-2017                         2273097
DrDoS_LDAP-2019                     2179930
LDAP-2019                           1915122
DrDoS_NTP-2019                      1202642
UDP-lag-2019                         366461
DoS Hulk-2017                        231073
Portmap-2019                         186960
PortScan-2017                        158930
DDoS-2017                            128027
BENIGN-2019                          113828
DoS GoldenEye-2017                    10293
FTP-Patator-2017          

Balance between benign and malicious classes.

In [7]:
set_seed()
df_balanced = balance_classes(df, ["BENIGN-2017", "BENIGN-2019"])

In [8]:
df_balanced[LABEL].value_counts()

Label
BENIGN-2017                        2273097
DrDoS_NetBIOS-2019                  123421
UDP-lag-2019                        123421
UDP-2019                            123421
TFTP-2019                           123421
Portmap-2019                        123421
PortScan-2017                       123421
NetBIOS-2019                        123421
MSSQL-2019                          123421
LDAP-2019                           123421
DrDoS_SSDP-2019                     123421
DrDoS_UDP-2019                      123421
DrDoS_NTP-2019                      123421
DrDoS_MSSQL-2019                    123421
DDoS-2017                           123421
DrDoS_LDAP-2019                     123421
DrDoS_DNS-2019                      123421
DoS Hulk-2017                       123421
DrDoS_SNMP-2019                     123420
Syn-2019                            123420
BENIGN-2019                         113828
DoS GoldenEye-2017                   10293
FTP-Patator-2017                      7938
SSH-P

Remove columns that do not appear in both the two datasets, and remove the additionally specified ones.

In [9]:
cols = set(pd.read_csv(files2017[0], index_col=0, skipinitialspace=True, nrows=0).columns.tolist()).intersection(
    pd.read_csv(files2019[0], index_col=0, skipinitialspace=True, nrows=0).columns.tolist())
[cols.discard(x) for x in EXCLUDED_COLUMNS]
cols = list(cols)
len(cols)

78

Load original samples from the files.

In [ ]:
final = []
for x in files2019 + files2017:
    tmp = dict.fromkeys((df_balanced[df_balanced['File'] == x]['Indexes'] + 1).tolist() + [0])
    tmp_df = pd.read_csv(x, index_col=0, skipinitialspace=True, usecols=cols, skiprows=lambda x: x not in tmp)
    final.append(tmp_df)
final = pd.concat(final, ignore_index=True)

In [15]:
final["Label"].value_counts()

Label
BENIGN                        2386925
DrDoS_DNS                      123421
DrDoS_SSDP                     123421
DDoS                           123421
DoS Hulk                       123421
MSSQL                          123421
Portmap                        123421
NetBIOS                        123421
UDP                            123421
DrDoS_NetBIOS                  123421
LDAP                           123421
DrDoS_UDP                      123421
DrDoS_LDAP                     123421
UDP-lag                        123421
TFTP                           123421
DrDoS_NTP                      123421
PortScan                       123421
DrDoS_MSSQL                    123421
Syn                            123420
DrDoS_SNMP                     123420
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
UDPLag

Transform all the possible features in numeric. This is useful in case the *read_csv* function was not able to correctly detect the data type when loading the *CSV*.

In [5]:
df_numeric = final.apply(pd.to_numeric, errors='ignore')

Converting remaining categorical features (e.g., HTTP method, GET, POST) into numerical, if any.

In [ ]:
cat_columns = [col_name for col_name,
        dtype in df_numeric.dtypes.items() if dtype == object and col_name not in ("Label", "Source")]
if cat_columns:
    print("Converting following categorical to numerical", cat_columns)
    df_numeric[cat_columns] = df_numeric[cat_columns].astype('category')
    df_numeric[cat_columns] = df_numeric[cat_columns].apply(lambda x: x.cat.codes)
    df_numeric[cat_columns] = df_numeric[cat_columns].astype('int')

Removing rows with missing values (e.g., NaN or Inf).

In [8]:
print("Shape before dropping NaN", df_numeric.shape)
df_wo_nan = df_numeric.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any")
print("Resulting shape", df_wo_nan.shape)

Shape before dropping NaN (4773850, 77)
Resulting shape (4705419, 77)


In [9]:
df_wo_nan["Label"].value_counts()

Label
BENIGN                        2384051
DDoS                           123419
PortScan                       123321
DrDoS_SNMP                     123163
DoS Hulk                       122924
DrDoS_NTP                      122662
DrDoS_UDP                      121819
DrDoS_SSDP                     121421
DrDoS_LDAP                     121251
UDP                            120957
LDAP                           120625
DrDoS_MSSQL                    119987
TFTP                           119953
DrDoS_NetBIOS                  119620
DrDoS_DNS                      119457
MSSQL                          119114
NetBIOS                        118664
Portmap                        116987
Syn                            113114
UDP-lag                        110995
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
UDPLag

Removing all the constant features among the selected dataset. These feature might result constant after the balancing of the samples, meaning that few samples with potentially different values for these features have been removed.

In [10]:
prevs = set(df_wo_nan.columns.values)
df_wo_const: pd.DataFrame = remove_constant_columns(df_wo_nan)
print("Remove constant removed the features", prevs - set(df_wo_const.columns.values))

Remove constant removed the features {'Bwd Avg Bulk Rate', 'Bwd Avg Packets/Bulk', 'Bwd URG Flags', 'Fwd Avg Bulk Rate', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', 'Bwd PSH Flags', 'Bwd Avg Bytes/Bulk'}


Remove identical features (cloned columns or same distribution). *Fwd Header Length.1* is an exact copy of *Fwd Header Length*, while the others present the same values of other columns in the dataset, hence only one is kept.

In [12]:
prevs = set(df_wo_const.columns.values)
df_wo_dup = df_wo_const[df_wo_const.describe(include="all").T.drop_duplicates().T.columns]
print("Identical columns removed", prevs - set(df_wo_dup.columns.values))

Identical columns removed {'Subflow Fwd Packets', 'Fwd Header Length.1', 'Avg Fwd Segment Size', 'Subflow Bwd Packets'}


Format column and label names (remove extra white spaces, UNICODE characters if any, etc.).

In [14]:
df_formatted = format_dataframe_columns_values(format_dataframe_columns_names(df_wo_dup), "Label")
df_formatted.index.name = "ID"

Dump the dataset.

In [15]:
dump(df_formatted, OUTPUT_DIR + "dataset.h5")